In [1]:
%matplotlib inline
from preamble import *
plt.rcParams['image.cmap'] = "gray"

ImportError: No module named preamble

### 4.6 Utilizing Expert Knowledge
- 도메인 전문가는 초기 데이터에서 더 유용한 특성을 선별하는 데 많은 도움을 줄 수 있음
- 뉴욕시티 자전거 대여 데이터를 사용한 문제
  - https://www.citibikenyc.com/system-data
  - 문제
    - 데이터: 2015년 8월 한달 동안에 대하여 3시간 단위로 앤디 집 앞에 있는 자전거 대여 횟수b
    - 특정 날짜와 시간에 앤디 집 앞에 있는 자전거를 사람들이 얼마나 대여하는지 예측
    - 앤디가 빌릴 수 있는 자전거가 남아 있는지 예측

In [2]:
citibike = mglearn.datasets.load_citibike()
print("citibike.data.shape:", citibike.shape)
print(type(citibike))

NameError: name 'mglearn' is not defined

In [3]:
print("Citibike data:\n{}".format(citibike.head()))

NameError: name 'citibike' is not defined

- 8월 한달 동안의 대여 횟수

In [4]:
plt.figure(figsize=(10, 3))

xticks = pd.date_range(
    start=citibike.index.min(), 
    end=citibike.index.max(),
    freq='D'
)

print(xticks)

NameError: name 'plt' is not defined

In [5]:
week = ["Sun", "Mon", "Tue", "Wed", "Thr", "Fri", "Sat"]
xtick_name = [week[int(w)] + d for w, d in zip(xticks.strftime("%w"), xticks.strftime(" %m-%d"))]

plt.xticks(xticks, xtick_name, rotation=90, ha="left")
plt.plot(citibike, linewidth=1)
plt.xlabel("Date")
plt.ylabel("Rentals")

NameError: name 'xticks' is not defined

- 24시간 간격으로 낮과 밤에 따라 자전거 대여 횟수에 확연한 차이가 있음
- 주중과 주말의 패턴도 차이가 있음
- 시계열 데이터를 이용한 예측 작업
  - 과거 데이터에서 학습하여 미래를 예측하는 방식
- 데이터 크기
  - 훈련 데이터: 23일 치 - 23 * 8 = 184개
  - 테스트 데이터: 8일 치 - 8 * 8 = 64
  - Note: 24 / 3 = 8

In [6]:
# extract the target values (number of rentals)
y = citibike.values
# convert to POSIX time by dividing by 10**9
X = citibike.index.astype("int64").values.reshape(-1, 1) // 10**9

NameError: name 'citibike' is not defined

In [7]:
print(X[0], y[0])
print(X[1], y[1])
print(X[2], y[2])
print()

print(X.shape)
print(y.shape)

NameError: name 'X' is not defined

In [8]:
# use the first 184 data points for training, the rest for testing
n_train = 184

# function to evaluate and plot a regressor on a given feature set
def eval_on_features(features, target, regressor):
    # split the given features into a training and a test set
    X_train, X_test = features[:n_train], features[n_train:]
    
    # also split the target array 
    y_train, y_test = target[:n_train], target[n_train:]

    regressor.fit(X_train, y_train)
    print("Test-set R^2: {:.2f}".format(regressor.score(X_test, y_test)))
    
    y_pred = regressor.predict(X_test)
    y_pred_train = regressor.predict(X_train)
    plt.figure(figsize=(10, 3))

    plt.xticks(range(0, len(X), 8), xticks.strftime("%a %m-%d"), rotation=90, ha="left")

    plt.plot(range(n_train), y_train, label="train")
    plt.plot(range(n_train, len(y_test) + n_train), y_test, '-', label="test")
    plt.plot(range(n_train), y_pred_train, '--', label="prediction train")

    plt.plot(range(n_train, len(y_test) + n_train), y_pred, '--', label="prediction test")
    plt.legend(loc=(1.01, 0))
    plt.xlabel("Date")
    plt.ylabel("Rentals")

In [9]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
eval_on_features(X, y, regressor)

NameError: name 'X' is not defined

- 훈련 데이터에 대한 예측은 정확
- 테스트 데이터에 대한 예측은 실패
  - $R^2$=-0.04
  - 테스트 데이터들은 훈련 데이터보다 모두 뒤에 있음
  - RandomForestRegressor는 훈련 데이터 특성 범위 밖으로 Extrapolation할 수 없음
    
- 해결 방법 (전문가 지식)
  - 훈련 데이터 특성에서 날짜 정보 (시간의 흐름 정보) 제외
  - 우선 시각 정보만 활용 (3시간 단위)

In [11]:
X_hour = citibike.index.hour.values.reshape(-1, 1)
for i in range(len(X_hour)):
    print(X_hour[i], end=", ")
print()
print()
print(X_hour.shape)
print()
eval_on_features(X_hour, y, regressor)

SyntaxError: invalid syntax (<ipython-input-11-ba7abdb74012>, line 3)

- 위 결과에서 시각에 따른 패턴 예측은 비교적 정확
  - $R^2$=0.6
  - 하지만, 주간 및 주말에 따른 패턴은 예측하지 못하고 있음<br/><br/>
  
 
- 해결 방법
  - 요일 정보를 훈련 데이터에 추가

In [12]:
a = citibike.index.dayofweek.values.reshape(-1, 1)
b = citibike.index.hour.values.reshape(-1, 1)
print(a.shape)
print(b.shape)
print()

X_hour_week = np.hstack([citibike.index.dayofweek.values.reshape(-1, 1),
                         citibike.index.hour.values.reshape(-1, 1)])

for i in range(len(X_hour_week)):
    print(X_hour_week[i], end=", ")
print()
print()
print(X_hour_week.shape)
print()

eval_on_features(X_hour_week, y, regressor)

SyntaxError: invalid syntax (<ipython-input-12-58efd832e3dd>, line 11)

- 예측의 결과가 비교적 정확
  - $R^2$=0.84

- RandomForestRegressor가 아닌 선형 모델 사용

In [13]:
from sklearn.linear_model import LinearRegression
eval_on_features(X_hour_week, y, LinearRegression())

NameError: name 'X_hour_week' is not defined

- LinearRegression 모델의 학습 결과가 좋지 않음
  - 요일과 시간이 정수로 인코딩되어 있어서 자동으로 연속형 변수로 해석됨
  - 시간을 선형 함수로만 학습하기 때문에 하루에서 시간이 흐를수록 결과값이 증가됨
  - 훈련 데이터 특성을 이산형으로 변형
     - OneHotEncorder 사용

In [14]:
enc = OneHotEncoder()
X_hour_week_onehot = enc.fit_transform(X_hour_week).toarray()

print(X_hour_week_onehot[0])
print(X_hour_week_onehot[1])
print(X_hour_week_onehot[2])
print()
print(X_hour_week_onehot.shape)
print()

NameError: name 'OneHotEncoder' is not defined

In [15]:
eval_on_features(X_hour_week_onehot, y, Ridge())

NameError: name 'X_hour_week_onehot' is not defined

In [16]:
poly_transformer = PolynomialFeatures(degree=2, interaction_only=True,
                                      include_bias=False)
X_hour_week_onehot_poly = poly_transformer.fit_transform(X_hour_week_onehot)
lr = Ridge()
eval_on_features(X_hour_week_onehot_poly, y, lr)

NameError: name 'PolynomialFeatures' is not defined

In [17]:
hour = ["%02d:00" % i for i in range(0, 24, 3)]
day = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
features =  day + hour

In [18]:
features_poly = poly_transformer.get_feature_names(features)
features_nonzero = np.array(features_poly)[lr.coef_ != 0]
coef_nonzero = lr.coef_[lr.coef_ != 0]

NameError: name 'poly_transformer' is not defined

In [19]:
plt.figure(figsize=(15, 2))
plt.plot(coef_nonzero, 'o')
plt.xticks(np.arange(len(coef_nonzero)), features_nonzero, rotation=90)
plt.xlabel("Feature name")
plt.ylabel("Feature magnitude")

NameError: name 'plt' is not defined